In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kinesis import KinesisUtils, InitialPositionInStream
from pyspark.sql.functions import col,regexp_extract,concat,lit,array,unix_timestamp,when,row_number
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.sql import SparkSession
import json 
from pyspark.ml.feature import StringIndexer,StringIndexerModel
from pyspark.conf import SparkConf
from rule_base.rules import Apache_error,Dnsmasq

import time

### For Phase 4 ###
from pyspark.sql.functions import from_unixtime, col, unix_timestamp, expr,lag,round,dayofweek,length,regexp_replace,count, desc,udf
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
import joblib

In [2]:
data=[
'{"time":"Jan 21 00:01:31","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"reply 3x6-.601-.TtfFB8j218UXwO8pVc68oxPnxxXW/9Otff-.Sipro*dGQOOMDr0IY1r2WRpfKdpesdSo9G-.i0rOSwEM5h8DYTFzpxaiOTWPKRKJaPZ4Yt-.ynQPRqKPCFFdlw6fN6NAz3VT516DMTVkCG-.customers_2017.xlsx.email-19.kennedy-mendoza.info is 195.128.194.168"}',
'{"time":"Jan 21 00:01:44","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"query[A] 3x6-.602-.OgEWPzRasaBfzw*1YgPwvfpK5ndYxfeL6t-.AAWCJrJfzqWHDvNtdSStYg3cnHaa0NHOxM-.26JBAO6h3xFvABN5REohh3RObsbew4lIQs-.XDxHdPvCAsmEqutMHFanKav*p1u36CvjZ4-.customers_2017.xlsx.email-19.kennedy-mendoza.info from 10.143.0.103"}',
'{"time":"Jan 21 00:01:44","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"forwarded 3x6-.602-.OgEWPzRasaBfzw*1YgPwvfpK5ndYxfeL6t-.AAWCJrJfzqWHDvNtdSStYg3cnHaa0NHOxM-.26JBAO6h3xFvABN5REohh3RObsbew4lIQs-.XDxHdPvCAsmEqutMHFanKav*p1u36CvjZ4-.customers_2017.xlsx.email-19.kennedy-mendoza.info to 192.168.231.254"}',
'{"time":"Jan 21 00:01:44","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"reply 3x6-.602-.OgEWPzRasaBfzw*1YgPwvfpK5ndYxfeL6t-.AAWCJrJfzqWHDvNtdSStYg3cnHaa0NHOxM-.26JBAO6h3xFvABN5REohh3RObsbew4lIQs-.XDxHdPvCAsmEqutMHFanKav*p1u36CvjZ4-.customers_2017.xlsx.email-19.kennedy-mendoza.info is 195.128.194.168"}',
'{"time":"Jan 21 00:02:03","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"query[A] 3x6-.603-.JAiihufiH6QIEbwkSuO95WxdELeD1lhhZM-.d9flhBVmZjVw*hHY4zQbeBvDhO7Rkx310e-.qpg3bNc2PfFMkhRJ5fHiwXm/77ZJqwhnpT-.wB1zoS/YSaMDXtmgJxN1OFHSuQqg51iWtQ-.customers_2017.xlsx.email-19.kennedy-mendoza.info from 10.143.0.103"}',
'{"time":"Jan 21 00:02:03","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"forwarded 3x6-.603-.JAiihufiH6QIEbwkSuO95WxdELeD1lhhZM-.d9flhBVmZjVw*hHY4zQbeBvDhO7Rkx310e-.qpg3bNc2PfFMkhRJ5fHiwXm/77ZJqwhnpT-.wB1zoS/YSaMDXtmgJxN1OFHSuQqg51iWtQ-.customers_2017.xlsx.email-19.kennedy-mendoza.info to 192.168.231.254"}',
'{"time":"Jan 21 00:02:03","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"reply 3x6-.603-.JAiihufiH6QIEbwkSuO95WxdELeD1lhhZM-.d9flhBVmZjVw*hHY4zQbeBvDhO7Rkx310e-.qpg3bNc2PfFMkhRJ5fHiwXm/77ZJqwhnpT-.wB1zoS/YSaMDXtmgJxN1OFHSuQqg51iWtQ-.customers_2017.xlsx.email-19.kennedy-mendoza.info is 195.128.194.168"}',
'{"time":"Jan 21 00:02:18","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"query[A] 3x6-.604-.MjQIsKRLJeIf3y1wM8tvXuGtFrV3H9WyLp-.FM74lLN5B5lFdxKv/oKEQF1IcKHe4qnp15-.myRWf8hQktgRZYoYMN84ec*T7Tx4Cu0F*5-.Atajwu7v2GXv2WXGRPV6jPKi6tEHcp72sD-.customers_2017.xlsx.email-19.kennedy-mendoza.info from 10.143.0.103"}',
'{"time":"Jan 21 00:02:18","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"forwarded 3x6-.604-.MjQIsKRLJeIf3y1wM8tvXuGtFrV3H9WyLp-.FM74lLN5B5lFdxKv/oKEQF1IcKHe4qnp15-.myRWf8hQktgRZYoYMN84ec*T7Tx4Cu0F*5-.Atajwu7v2GXv2WXGRPV6jPKi6tEHcp72sD-.customers_2017.xlsx.email-19.kennedy-mendoza.info to 192.168.231.254"}',
'{"time":"Jan 21 00:02:18","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"reply 3x6-.604-.MjQIsKRLJeIf3y1wM8tvXuGtFrV3H9WyLp-.FM74lLN5B5lFdxKv/oKEQF1IcKHe4qnp15-.myRWf8hQktgRZYoYMN84ec*T7Tx4Cu0F*5-.Atajwu7v2GXv2WXGRPV6jPKi6tEHcp72sD-.customers_2017.xlsx.email-19.kennedy-mendoza.info is 195.128.194.168"}',
'{"time":"Jan 21 00:02:27","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"query[TXT] current.cvd.clamav.net from 172.19.130.4"}',
'{"time":"Jan 21 00:02:27","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"forwarded current.cvd.clamav.net to 192.168.231.254"}',
'{"time":"Jan 21 00:02:27","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"reply current.cvd.clamav.net is 0.103.5:62:26428:1642717740:1:90:49192:333"}',
'{"time":"Jan 21 00:02:27","ident":"dnsmasq","owner":"client01","log_format":"dnsmasq","pid":"3468","message":"query[A] db.local.clamav.net from 172.19.130.4"}'
]


In [3]:
conf = (
    SparkConf()
    .setAppName("KInesisTest") \
    .set("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .set("spark.hadoop.fs.s3a.access.key", "ACCESS_KEY") \
    .set("spark.hadoop.fs.s3a.secret.key", "SECRET_ACCESS_KEY") \
)

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

In [35]:
dnsmasq_regex = r"([a-z]+\[?[A-Z]*\]?)\s(\S+)\s([fromtois]+)\s(\S+)"
indexer_dnsmasq = StringIndexerModel.read().load("./model/indexer_dnsmasq")
indexer_error = StringIndexerModel.read().load("./model/indexer_apacheerror")
def process_dnsmasq(filtered_rdd):
    #fix timestamp, create key, create value
    df=spark.createDataFrame(filtered_rdd)
    df=df.withColumn("response", regexp_extract("message", dnsmasq_regex, 1)) \
    .withColumn("domain", regexp_extract("message", dnsmasq_regex, 2)) \
    .withColumn("term", regexp_extract("message", dnsmasq_regex, 3)) \
    .withColumn("ip_addr", regexp_extract("message", dnsmasq_regex, 4)) \
    .withColumn("key", concat("response", lit(" <*> "),"term", lit(" <*>"))) \
    .withColumn("value1", regexp_extract("message", dnsmasq_regex, 2)) \
    .withColumn("value2", regexp_extract("message", dnsmasq_regex, 4)) \
    .withColumn("time", concat("time", lit(" 2022"))) \
    .withColumn("epoch_timestamp", unix_timestamp("time", "MMM dd HH:mm:ss yyyy")) 
    df_indexed=indexer_dnsmasq.transform(df)
    df_indexed=df_indexed.drop("pid","time","domain","term","ip_addr")
    return df_indexed

In [36]:
def categorize_ip(value2):
    try:
        # Split IP address into octets
        octets = list(map(int, value2.split('.')))

        # Check if IP is private
        if (octets[0] == 10) or (octets[0] == 172 and 16 <= octets[1] <= 31) or (octets[0] == 192 and octets[1] == 168):
            return 0  # Private IP
        else:
            return 1  # Public IP
    except:
        return 2  # Non-IP values
    
# Custom function to count dots
def count_dots(s):
    return s.count('.')

# Custom function to count hyphens
def count_hyphens(s):
    return s.count('-')

In [37]:
def process_dnsmasq_for_pred(df_pyspark):
    # df_pyspark = df_pyspark.drop("ident", "log_format", "owner", "message")  # Remove unnecessary columns
    df_pyspark = df_pyspark.withColumn("encoded_key", col("encoded_key").cast("int"))
    df_pyspark = df_pyspark.withColumn("timestamp_datetime", from_unixtime("epoch_timestamp").cast("timestamp"))

    lag_col = lag(col("epoch_timestamp")).over(Window.orderBy("epoch_timestamp"))
    df_pyspark = df_pyspark.withColumn("time_diff_unix", round((col("epoch_timestamp") - lag_col), 1))
    df_pyspark = df_pyspark.fillna(0, subset=["time_diff_unix"])
    df_pyspark = df_pyspark.withColumn("time_diff_unix", col("time_diff_unix").cast("decimal(10,1)"))

    df_pyspark = df_pyspark.withColumn("day_of_week", dayofweek("timestamp_datetime"))
    
    df_pyspark = df_pyspark.withColumn("value1_length", length("value1"))
    df_pyspark = df_pyspark.withColumn("value2_length", length("value2"))

    count_dots_udf = udf(count_dots, IntegerType())
    count_hyphens_udf = udf(count_hyphens, IntegerType())
    

    ##### new 

    # Apply UDFs to DataFrame
    df_pyspark = df_pyspark.withColumn("value1_dot_count", count_dots_udf("value1"))
    df_pyspark = df_pyspark.withColumn("value1_hyphen_count", count_hyphens_udf("value1"))
    df_pyspark = df_pyspark.withColumn("value2_dot_count", count_dots_udf("value2"))
    df_pyspark = df_pyspark.withColumn("value2_hyphen_count", count_hyphens_udf("value2"))

    ##### new 

    df_pyspark = df_pyspark.withColumn("key_length", length("key"))

    window_spec_value1 = Window().orderBy("value1")
    window_spec_value2 = Window().orderBy("value2")
    
    df_pyspark = df_pyspark.withColumn("value1_count", count("value1").over(window_spec_value1))
    df_pyspark = df_pyspark.withColumn("value2_count", count("value2").over(window_spec_value2))

    # Register the UDF
    categorize_ip_udf = udf(categorize_ip, IntegerType())
    
    # Apply the UDF to create a new column 'ip_category'
    df_pyspark = df_pyspark.withColumn("value2_ip_class", categorize_ip_udf("value2"))
    
    return df_pyspark

In [69]:

def process_rdd(rdd):
    print("enter check")
    start_time_process = time.time()
    if not rdd.isEmpty():
        print("not empty")
        dataframes = {}
        json_rdd = rdd.map(lambda x: json.loads(x))
        apache_error = json_rdd.filter(lambda x: x.get('log_format') == 'apache_error')
        dnsmasq_rdd = json_rdd.filter(lambda x: x.get('log_format') == 'dnsmasq')
        apache2_access=json_rdd.filter(lambda x:x.get('log_format')=="apache2_access")
        if not dnsmasq_rdd.isEmpty():
            print("***** phase 1 dnsmasq log seperation ******")
            
            start_time_dns = time.time()

            dataframes['dnsmasq'] = process_dnsmasq(dnsmasq_rdd)
            dataframes['dnsmasq'].show()
            unique_owners=dataframes['dnsmasq'].select('owner').distinct()
            
            end_time_dns = time.time()

            elapsed_time = end_time_dns - start_time_dns
            print("Pre-processed time <dnsmasq_phase_1>:", elapsed_time, "seconds\n")
            log_model = joblib.load('C:\\Users\\A570ZD\\Desktop\\siem dev2\\model\\ML_trained_model\\RandomForestClassifier2.joblib')
            
            log_model.feature_names = None

            broadcast_model = sc.broadcast(log_model)
            @udf('integer')
            def predict_data(*cols):
                return int(broadcast_model.value.predict((cols,)))
            
            for row in unique_owners.collect():
                    print("***** phase 2 dnsmasq owner seperation & furthur pre-processing ******")

                    # since only 1 column is collected , so it's always at row[0]
                    unique_value = row[0]
                    df_temp = dataframes['dnsmasq'].filter(dataframes['dnsmasq']['owner'] == unique_value)
                    owner = df_temp.select("owner").first()[0]
                    # df_temp.show()

                    df_temp = process_dnsmasq_for_pred(df_temp)  
                    # df_temp.show() 

                    end_time_dns = time.time()

                    elapsed_time = end_time_dns - start_time_dns
                    print(f"Pre-processed time <dnsmasq_phase_2_{owner}>:", elapsed_time, "seconds\n")
                    
                    print("***** phase 3 dnsmasq  rule-base detection ******")
                    start_time_dns = time.time()

                    ##### code for rule based prediction
                    
                    print("XXXX nothing XXXX")

 

                    # send to anomaly module
                    print("***** phase 4 dnsmasq  anomaly detection ******")
                    

                    df_pyspark = df_temp.alias("df_pyspark")
                    # df_pyspark = df_pyspark.drop("key", "value1", "value2", "epoch_timestamp","timestamp_datetime")
                    df_pyspark = df_pyspark.selectExpr(
                        'encoded_key',
                        'time_diff_unix',
                        'day_of_week',
                        'value1_length',
                        'value2_length',
                        'value1_dot_count',
                        'value1_hyphen_count',
                        'value2_dot_count',
                        'value2_hyphen_count',
                        'key_length',
                        'value1_count',
                        'value2_count',
                        'value2_ip_class'  # Assuming this column needs to be added
                    )

                    list_of_columns = df_pyspark.columns
                    df_pyspark = df_pyspark.withColumn("prediction", predict_data(*list_of_columns))
                    prediction_column=df_pyspark.select("prediction")
                    # df_temp=df_temp.join(prediction_column)
                    w = Window().orderBy(lit('A'))
                    prediction_column = prediction_column.withColumn('id', row_number().over(w))
                    df_temp = df_temp.withColumn('id', row_number().over(w))

                    #join together both DataFrames using 'id' column
                    df_temp = df_temp.join(prediction_column, on=['id']).drop('id')
                    df_temp=df_temp.select('response','value1',"value2","prediction")
                    df_pyspark.show()

                    end_time_dns = time.time()

                    elapsed_time = end_time_dns - start_time_dns
                    
                    print(f"Anomaly detection time <dnsmasq_phase_4_{owner}>:", elapsed_time, "seconds\n")
                    print("******** phase 5 rule creation **********")
                    


                    ########################
                    print("************ END ***********************************") 
                    return df_temp,unique_value
                    

In [61]:
rdd = spark.sparkContext.parallelize(data)

In [62]:
df_anomaly_extraact,uniue_value=process_rdd(rdd)

enter check
not empty
***** phase 1 dnsmasq log seperation ******
+-------+----------+--------------------+--------+----------+--------------------+--------------------+--------------------+---------------+-----------+
|  ident|log_format|             message|   owner|  response|                 key|              value1|              value2|epoch_timestamp|encoded_key|
+-------+----------+--------------------+--------+----------+--------------------+--------------------+--------------------+---------------+-----------+
|dnsmasq|   dnsmasq|reply 3x6-.601-.T...|client01|     reply|    reply <*> is <*>|3x6-.601-.TtfFB8j...|     195.128.194.168|     1642698091|        0.0|
|dnsmasq|   dnsmasq|query[A] 3x6-.602...|client01|  query[A]|query[A] <*> from...|3x6-.602-.OgEWPzR...|        10.143.0.103|     1642698104|        2.0|
|dnsmasq|   dnsmasq|forwarded 3x6-.60...|client01| forwarded|forwarded <*> to <*>|3x6-.602-.OgEWPzR...|     192.168.231.254|     1642698104|        1.0|
|dnsmasq|   dnsm

In [63]:
df_anomaly_extraact.show()

+----------+--------------------+--------------------+----------+
|  response|              value1|              value2|prediction|
+----------+--------------------+--------------------+----------+
|     reply|3x6-.601-.TtfFB8j...|     195.128.194.168|         0|
|  query[A]|3x6-.602-.OgEWPzR...|        10.143.0.103|         1|
| forwarded|3x6-.602-.OgEWPzR...|     192.168.231.254|         1|
|     reply|3x6-.602-.OgEWPzR...|     195.128.194.168|         1|
|  query[A]|3x6-.603-.JAiihuf...|        10.143.0.103|         0|
| forwarded|3x6-.603-.JAiihuf...|     192.168.231.254|         0|
|     reply|3x6-.603-.JAiihuf...|     195.128.194.168|         1|
|  query[A]|3x6-.604-.MjQIsKR...|        10.143.0.103|         1|
| forwarded|3x6-.604-.MjQIsKR...|     192.168.231.254|         1|
|     reply|3x6-.604-.MjQIsKR...|     195.128.194.168|         0|
|query[TXT]|current.cvd.clama...|        172.19.130.4|         1|
| forwarded|current.cvd.clama...|     192.168.231.254|         1|
|     repl

In [64]:
rule_df=df_anomaly_extraact[df_anomaly_extraact['prediction']==1].drop('prediction')

In [65]:
rule_df.show()

+----------+--------------------+--------------------+
|  response|              value1|              value2|
+----------+--------------------+--------------------+
|  query[A]|3x6-.602-.OgEWPzR...|        10.143.0.103|
| forwarded|3x6-.602-.OgEWPzR...|     192.168.231.254|
|     reply|3x6-.602-.OgEWPzR...|     195.128.194.168|
|     reply|3x6-.603-.JAiihuf...|     195.128.194.168|
|  query[A]|3x6-.604-.MjQIsKR...|        10.143.0.103|
| forwarded|3x6-.604-.MjQIsKR...|     192.168.231.254|
|query[TXT]|current.cvd.clama...|        172.19.130.4|
| forwarded|current.cvd.clama...|     192.168.231.254|
|     reply|current.cvd.clama...|0.103.5:62:26428:...|
|  query[A]| db.local.clamav.net|        172.19.130.4|
+----------+--------------------+--------------------+



In [66]:
fltered_df = rule_df.filter((rule_df["response"].like("%query%"))).select("value2").distinct()

In [57]:
malicious_ip=fltered_df.rdd.flatMap(lambda x:x).collect()

In [58]:
print(malicious_ip)

['10.143.0.103', '172.19.130.4']


In [68]:
fltered_df.write.csv("./rule_base/{unique_value}/malicious_ip.csv".format(unique_value=uniue_value),mode="append")

a


In [25]:
def read_txt_to_list(file_path):
    lines_list = []
    with open(file_path, 'r') as file:
        for line in file:
            lines_list.append(line.strip())  # strip() removes any leading/trailing whitespaces including '\n'
    return lines_list

# Example usage:
file_path = 'dnsmasq_sample_chop.txt'  # Replace 'example.txt' with the path to your text file
lines = read_txt_to_list(file_path)
print(lines)
print('\n', lines[-1])



['{"time":"Jan 21 00:00:09","ident":"dnsmasq","owner":"client02","log_format":"dnsmasq","pid":"3468","message":"query[A] 3x6-.596-.IunWTzebVlyAhhHj*ZfWjOBun1zAf*Wgpq-.YarqcF7oovex5JXZQp35nThgDU1Q3p3lT/-.DM6Vx/vcq3AkrO4Xh2kjojk8RCiDE2wjSv-.gY6ONv8eNmDck8gGwJ8fU3PPctbthfeDZT-.customers_2017.xlsx.email-19.kennedy-mendoza.info from 10.143.0.103"}', '{"time":"Jan 21 00:00:09","ident":"dnsmasq","owner":"client02","log_format":"dnsmasq","pid":"3468","message":"forwarded 3x6-.596-.IunWTzebVlyAhhHj*ZfWjOBun1zAf*Wgpq-.YarqcF7oovex5JXZQp35nThgDU1Q3p3lT/-.DM6Vx/vcq3AkrO4Xh2kjojk8RCiDE2wjSv-.gY6ONv8eNmDck8gGwJ8fU3PPctbthfeDZT-.customers_2017.xlsx.email-19.kennedy-mendoza.info to 192.168.231.254"}', '{"time":"Jan 21 00:00:09","ident":"dnsmasq","owner":"client02","log_format":"dnsmasq","pid":"3468","message":"reply 3x6-.596-.IunWTzebVlyAhhHj*ZfWjOBun1zAf*Wgpq-.YarqcF7oovex5JXZQp35nThgDU1Q3p3lT/-.DM6Vx/vcq3AkrO4Xh2kjojk8RCiDE2wjSv-.gY6ONv8eNmDck8gGwJ8fU3PPctbthfeDZT-.customers_2017.xlsx.email-19.kenne